In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Vasco\Documents\openslide-win64-20231011\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [8]:
pth = r'\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist'  # path of ndpi files

pth_mat = r'\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist\1x\matfiles'  # path .mat files with crop data (got from matlab file)

mat_files = [_ for _ in os.listdir(pth_mat) if _.endswith('mat')]
WSIs = [_ for _ in os.listdir(pth) if _.endswith('ndpi')]

In [9]:
crop_mats

['crop_fallopian_tubes_AJER376_035.mat',
 'crop_fallopian_tubes_AJER376_211.mat',
 'crop_fallopian_tubes_AJER376_431.mat',
 'crop_fallopian_tubes_AJGB283_155.mat',
 'crop_fallopian_tubes_AJGB283_487.mat',
 'crop_fallopian_tubes_AJLC170_0153.mat',
 'crop_fallopian_tubes_AJLC170_0729.mat',
 'crop_fallopian_tubes_AJLF232_0153.mat',
 'crop_fallopian_tubes_AJLF232_0483.mat']

In [10]:
WSIs

['fallopian_tubes_AJER376_035.ndpi',
 'fallopian_tubes_AJER376_211.ndpi',
 'fallopian_tubes_AJER376_431.ndpi',
 'fallopian_tubes_AJGB283_155.ndpi',
 'fallopian_tubes_AJGB283_487.ndpi',
 'fallopian_tubes_AJLC170_0153.ndpi',
 'fallopian_tubes_AJLC170_0729.ndpi',
 'fallopian_tubes_AJLF232_0153.ndpi',
 'fallopian_tubes_AJLF232_0483.ndpi']

In [11]:
res20x=0.4416
res1x=8

ratio = res1x/res20x

outpth = os.path.join(pth,'tiles512')
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [12]:
outpth

'\\\\10.99.68.178\\andreex\\students\\Vasco Fallopian tube\\stardist\\tiles512'

In [17]:
from scipy.io import loadmat
import h5py

file_format = '.tif'

for i, file in enumerate(mat_files):
    mat_file_name = os.path.join(pth_mat, file)
    # print(mat_file_name)
    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(mat_files)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(mat_files)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]
        crop_width = 512
        crop_height = 512
        #data = loadmat(mat_file_name)
        try:
            data = loadmat(mat_file_name)

            crop_x = int(data['crop_x']*ratio)
            crop_y = int(data['crop_y']*ratio)
        except:
            print(mat_file_name)
            data = h5py.File(mat_file_name, 'r')

            crop_x = int(data['crop_x'][()]*ratio)
            crop_y = int(data['crop_y'][()]*ratio)
        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)

        # print(data['crop_x'])


        lower_x = crop_x
        upper_y = crop_y
        # upper_x = crop_x + crop_width
        # lower_y = crop_y - crop_height

        print(crop_x)
        print(crop_y)
        # print(crop_width)
        # print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width,crop_height)) # level = 0 in middle
        # region = WSI_slide.read_region((crop_y, crop_x), 0, (crop_width, crop_height)) # level = 0 in middle

        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()



Skipping cropping for fallopian_tubes_AJER376_035.ndpi (1/9)... Already exists
Skipping cropping for fallopian_tubes_AJER376_211.ndpi (2/9)... Already exists
Skipping cropping for fallopian_tubes_AJER376_431.ndpi (3/9)... Already exists
Skipping cropping for fallopian_tubes_AJGB283_155.ndpi (4/9)... Already exists
Skipping cropping for fallopian_tubes_AJGB283_487.ndpi (5/9)... Already exists
Skipping cropping for fallopian_tubes_AJLC170_0153.ndpi (6/9)... Already exists
Skipping cropping for fallopian_tubes_AJLC170_0729.ndpi (7/9)... Already exists
Skipping cropping for fallopian_tubes_AJLF232_0153.ndpi (8/9)... Already exists
Cropping fallopian_tubes_AJLF232_0483.ndpi (9/9)
33668
29655


In [14]:
f_out_name

'\\\\10.99.68.178\\andreex\\students\\Vasco Fallopian tube\\stardist\\tiles512\\fallopian_tubes_AJLF232_0483.tif'